In [1]:
!pip install catboost
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [4]:
# !python make_dataset_top4.py

In [5]:
from make_dataset_top4 import get_dateset
train_df,val_df = get_dateset(save=True)

finish make val_data
finish make data


In [6]:
def getXy(df):
    X = df.iloc[:,:-55]
    y = df.iloc[:,-55:]
    return X,y

In [7]:
X_train,y_train=getXy(train_df)
X_train.head()

,dnn1_0,dnn1_1,dnn1_2,dnn1_3,dnn1_4,dnn1_5,dnn1_6,dnn1_7,dnn1_8,dnn1_9,...,dnn2_1018,dnn2_1019,dnn2_1020,dnn2_1021,dnn2_1022,dnn2_1023,other_f_0,other_f_1,sex,age
0,0.659524,0.051887,0.170676,0.014643,0.021988,0.021581,0.077981,0.212418,0.249080,0.082551,...,10.105191,0.714911,0.473581,0.0,12.949687,1.365231,0,0,-999,-999
1,0.517988,0.024537,0.210841,0.005725,0.009323,0.016905,0.037280,0.176291,0.261812,0.047190,...,12.058694,0.974568,1.222527,0.0,15.146009,1.455690,0,0,57,0
2,0.665105,0.053404,0.160071,0.011156,0.011763,0.016511,0.095234,0.232191,0.169899,0.071827,...,10.899384,0.510629,0.755528,0.0,13.416583,1.467101,0,0,63,0
3,0.604943,0.063382,0.241103,0.021044,0.027337,0.025420,0.092307,0.265270,0.252515,0.107501,...,8.744452,0.723036,0.777904,0.0,10.240247,1.065250,0,0,23,0
4,0.530529,0.028505,0.242730,0.014185,0.032382,0.030528,0.036515,0.206008,0.284360,0.076480,...,8.684827,0.732461,1.138609,0.0,12.168888,1.086403,0,0,31,1


In [8]:
y_train.head()

,窦性心律,T波改变,窦性心动过缓,ST段改变,正常ECG,左心室高电压,ST-T改变,窦性心动过速,临界ECG,QRS低电压,...,右心房扩大,左束支传导阻滞,前间壁R波递增不良,右心室肥大,房室传导延缓,双分支传导阻滞,非特异性室内传导阻滞,肺型P波,完全性左束支传导阻滞,融合波
0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
import numpy as np
X_validation, y_validation=getXy(val_df)
cat_features = np.where(X_validation.dtypes != np.float)[0]#list(range( X_train.shape[1]-4, X_train.shape[1]))
cat_features

array([1079, 1080, 1081, 1082])

In [17]:
from catboost import CatBoostClassifier
model_list=[None]*y_train.shape[1]
for tagi,model in enumerate(model_list):
    if model_list[tagi]==None:
        model = CatBoostClassifier(
            iterations=10,
            random_seed=42,
            eval_metric='F1',
            learning_rate=0.03,#todo 超参数选择
    #         task_type='GPU',
            od_type='Iter',#早停
            od_wait= 40
        )
        model_list[tagi]=model
    
    model.fit(
        X_train, y_train.iloc[:,tagi],
        cat_features=cat_features,
        eval_set=(X_validation, y_validation.iloc[:,tagi]),
        verbose=False,#打印
        plot=False#作图
    )
    print(y_train.columns[tagi],model.get_best_score())


窦性心律 {'learn': {'Logloss': 0.5502412468660763, 'F1': 0.8969469625924767}, 'validation': {'Logloss': 0.553039232761245, 'F1': 0.8954228554107304}}
T波改变 {'learn': {'Logloss': 0.5230816895870729, 'F1': 0.00849256900212314}, 'validation': {'Logloss': 0.5212352410818599, 'F1': 0.0}}
窦性心动过缓 {'learn': {'Logloss': 0.34384881719725796, 'F1': 0.8004640371229699}, 'validation': {'Logloss': 0.34815317651667893, 'F1': 0.7804878048780488}}
ST段改变 {'learn': {'Logloss': 0.4765696605115915, 'F1': 0.0}, 'validation': {'Logloss': 0.49699763417309767, 'F1': 0.0}}
正常ECG {'learn': {'Logloss': 0.523949391024357, 'F1': 0.48880597014925375}, 'validation': {'Logloss': 0.5200458222866067, 'F1': 0.4929178470254958}}
左心室高电压 {'learn': {'Logloss': 0.49972548912128695, 'F1': 0.0049504950495049506}, 'validation': {'Logloss': 0.5209174551489275, 'F1': 0.0056022408963585435}}
ST-T改变 {'learn': {'Logloss': 0.41408194935787107, 'F1': 0.4427333974975938}, 'validation': {'Logloss': 0.41217463871916427, 'F1': 0.363057324840764

In [18]:
import numpy as np
# 预测
def model_list_predict(model_list,X):
    y_pre=[[]]*len(model_list)
    for tagi,model in enumerate(model_list):
        y_pre[tagi]=model.predict(X)
    y_pre=np.array(y_pre).T
    return y_pre

y_pred=model_list_predict(model_list,X_validation)

In [19]:
#F1
from sklearn.metrics import f1_score
def F1_metric(model_list,X,y):
    y_pred=model_list_predict(model_list,X)
    F1=f1_score(y,y_pred,average='micro')
    return F1
train_F1=F1_metric(model_list,X_train,y_train) 
val_F1=F1_metric(model_list,X_validation,y_validation) 
print('train_F1:%.4f\tval_F1:%.4f'%(train_F1,val_F1))

train_F1:0.5909	val_F1:0.5717


In [27]:
from catboost import CatBoostClassifier
model_list=[None]*y_train.shape[1]
max_epoch=10
for epoch in range(max_epoch):
    print('epoch\t%d'%epoch)
#     train_df,val_df = get_dateset(save=True)
#     X_train,y_train=getXy(train_df)
#     X_validation, y_validation=getXy(val_df)
#     if cat_features is None:
#         cat_features = np.where(X_validation.dtypes != np.float)[0]
    for tagi,model in enumerate(model_list):
        if model_list[tagi]==None:
            model = CatBoostClassifier(
                iterations=1,
                random_seed=42,
                eval_metric='F1',
                learning_rate=0.03,#todo 超参数选择
        #         task_type='GPU',
                od_type='Iter',#早停
                od_wait= 40
            )
            model_list[tagi]=model
            baseline=None
        else:
            baseline = model.predict(X_train, prediction_type='RawFormulaVal')
        model.fit(
            X_train, y_train.iloc[:,tagi],
            cat_features=cat_features,
#             eval_set=(X_validation, y_validation.iloc[:,tagi]),
            verbose=False,#打印
            plot=False,#作图
            baseline=baseline
        )
#         print('\t%s\ttrain_F1=%.4f\tval_F1=%.4f'%(y_train.columns[tagi],model.get_best_score()['learn']['F1'],model.get_best_score()['validation']['F1']))
    train_F1=F1_metric(model_list,X_train,y_train) 
    val_F1=F1_metric(model_list,X_validation,y_validation) 
    print('epoch=%d\ttrain_F1:%.4f\tval_F1:%.4f'%(epoch,train_F1,val_F1))

epoch	0
epoch=0	train_F1:0.5799	val_F1:0.5619
epoch	1
epoch=1	train_F1:0.5811	val_F1:0.5638
epoch	2
epoch=2	train_F1:0.5805	val_F1:0.5618
epoch	3
epoch=3	train_F1:0.5811	val_F1:0.5638
epoch	4
epoch=4	train_F1:0.5805	val_F1:0.5618
epoch	5
epoch=5	train_F1:0.5811	val_F1:0.5638
epoch	6
epoch=6	train_F1:0.5805	val_F1:0.5618
epoch	7
epoch=7	train_F1:0.5811	val_F1:0.5638
epoch	8
epoch=8	train_F1:0.5805	val_F1:0.5618
epoch	9
epoch=9	train_F1:0.5811	val_F1:0.5638


In [ ]:
model.save_model('ckpt/catboost_model.dump')
model = CatBoostClassifier()
model.load_model('ckpt/catboost_model.dump');